# データ・サイエンス	データ・サイエンス社会応用論 / ICT社会応用演習Ⅳ
## 11-4. GTFS Realtime : Trip Update

### 1. 下準備

In [ ]:
import requests
import zipfile
import os
from google.transit import gtfs_realtime_pb2
import pandas as pd
import geopandas as gpd

In [ ]:
AOMORI_BUS_ALERT_URL = 'https://api-public.odpt.org/api/v4/gtfs/realtime/odpt_AomoriCity_AllLines_alert'
AOMORI_BUS_TRIPUPDATE_URL = 'https://api-public.odpt.org/api/v4/gtfs/realtime/odpt_AomoriCity_AllLines_trip_update'
AOMORI_BUS_VEHICLEPOSITION_URL = 'https://api-public.odpt.org/api/v4/gtfs/realtime/odpt_AomoriCity_AllLines_vehicle'

### 2. GTFS Realtime のダウンロード
#### 2-a) フィードの読み込み

In [ ]:
response = requests.get(AOMORI_BUS_TRIPUPDATE_URL)

if response.status_code == 200:
    pb_data = response.content

In [ ]:
feed = gtfs_realtime_pb2.FeedMessage()
feed.ParseFromString(pb_data)

#### 2-b) ヘッダとエンティティの表示

In [ ]:
print(feed.header)

In [ ]:
for entity in feed.entity:
    print(entity)

#### 2-c) Pandas での読み込み

In [ ]:
data = []
for entity in feed.entity:
    if not entity.HasField('trip_update'):
        continue
    
    trip_update = entity.trip_update

    for stop_time_update in trip_update.stop_time_update:    
        data.append({
            'trip_id': trip_update.trip.trip_id,
            'stop_sequence': stop_time_update.stop_sequence,
            'delay': stop_time_update.departure.delay
        })

df_trip_update = pd.DataFrame(data)
df_trip_update

### 3. GTFSの読み込み
#### 3-a) GTFSのダウンロードと展開

In [ ]:
AOMORI_BUS_GTFS = 'https://api-public.odpt.org/api/v4/files/odpt/AomoriCity/AllLines.zip?date=20241201'

ZIP_PATH = 'tmp.zip'
TMPDIR_PATH = 'tmp/'

In [ ]:
response = requests.get(AOMORI_BUS_GTFS)

if response.status_code == 200:
  with open(ZIP_PATH, 'wb') as f:
    f.write(response.content)

if not os.path.exists(TMPDIR_PATH):
  os.makedirs(TMPDIR_PATH)

with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
  zip_ref.extractall(TMPDIR_PATH)

#### 3-b) stops.txt / routes.txt / trips.txt / stop_times.txt の読み込み

In [ ]:
df_stops = pd.read_csv(TMPDIR_PATH + 'stops.txt')
df_stops = df_stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']]
df_stops

In [ ]:
df_routes = pd.read_csv(TMPDIR_PATH + 'routes.txt')
df_routes = df_routes[['route_id', 'route_short_name', 'route_long_name']]
df_routes

In [ ]:
df_trips = pd.read_csv(TMPDIR_PATH + 'trips.txt')
df_trips = df_trips[['route_id', 'service_id', 'trip_id', 'trip_headsign']]
df_trips

In [ ]:
df_stop_times = pd.read_csv(TMPDIR_PATH + 'stop_times.txt')
df_stop_times = df_stop_times[['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence', 'stop_headsign']]
df_stop_times

#### 3-c) 特定の路線での絞り込み

In [ ]:
df_routes_a1 = df_routes[df_routes['route_short_name'] == 'A1 国道・古川線']
df_trips_a1 = df_routes_a1.merge(df_trips)
df_trips_a1

#### 3-d) 特定の路線上のルート更新情報を確認

In [ ]:
df_trip_update_a1 = df_trip_update.merge(df_trips_a1)
df_trip_update_a1

In [ ]:
df_stops_names = df_stops[['stop_id', 'stop_name']]
df_trip_update_a1.merge(df_stop_times).merge(df_stops_names)